In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tamber/steam-video-games")

print("Path to dataset files:", path)

100%|██████████| 1.46M/1.46M [00:00<00:00, 59.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/tamber/steam-video-games/versions/3


In [20]:
import pandas as pd

df = pd.read_csv(path + "/steam-200k.csv")

In [21]:
# columns 이름 변경 + 필요 없는 column 삭제
df = df.rename(columns={'151603712':'user_id', 'The Elder Scrolls V Skyrim': 'game_name', '1.0':'play_time'}).drop(columns=['0'])
df.head()

,user_id,game_name,purchase,play_time
0,151603712,The Elder Scrolls V Skyrim,play,273.0
1,151603712,Fallout 4,purchase,1.0
2,151603712,Fallout 4,play,87.0
3,151603712,Spore,purchase,1.0
4,151603712,Spore,play,14.9


In [24]:
df['user_id'].value_counts()

,count
user_id,
62990992,1573
33865373,949
11403772,906
30246419,901
47457723,855
...,...
245434963,1
255576471,1
296979942,1


In [30]:
# purchase만 한 수 확인
purchase = len(df[df['purchase']=='purchase'])
play = len(df[df['purchase']=='play'])
print('num of purchase: ', purchase, ', num of play: ', play)
print('purchase no play: ', purchase - play)

num of purchase:  129510 , num of play:  70489
purchase no play:  59021


In [34]:
# nan 유무 확인
df.isnull().sum()

,0
user_id,0
game_name,0
purchase,0
play_time,0


In [44]:
# 1시간 play한 경우 있는지 확인
exists = not df[(df['purchase'] == 'play') & (df['play_time'] == 1.0)].empty
print(exists)

True


In [45]:
# purchase와 1시간 play의 play time 구별 위해 play의 play time에 전체적으로 1을 더해주도록 함
df.loc[df['purchase'] == 'play', 'play_time'] += 1

In [46]:
exists = not df[(df['purchase'] == 'play') & (df['play_time'] == 1.0)].empty
print(exists)

False


## Utility matrix 생성 시작

In [47]:
game = df['game_name'].unique()
user = df['user_id'].unique()

# 일단 rating 다 Nan으로 시작
utility = pd.DataFrame(index=user, columns=game)

In [51]:
# 같은 user_id, game_name별로 가장 큰 play_time 선택
df_max_time = df.groupby(['user_id', 'game_name'])['play_time'].max().reset_index()

# utility matrix에 값 채워 넣기
for _, row in df_max_time.iterrows():
    utility.loc[row['user_id'], row['game_name']] = row['play_time']

print(utility.head())

          The Elder Scrolls V Skyrim Fallout 4 Spore Fallout New Vegas  \
151603712                      274.0      88.0  15.9              13.1   
187131847                        NaN       NaN   NaN               NaN   
59945701                        59.0       NaN   NaN               1.0   
53875128                         1.0       NaN   1.5               8.5   
234941318                        NaN       NaN   NaN               NaN   

          Left 4 Dead 2 HuniePop Path of Exile Poly Bridge Left 4 Dead  \
151603712           9.9      9.5           9.1         8.5         4.3   
187131847           NaN      NaN           NaN         NaN         NaN   
59945701            1.0      NaN           NaN         NaN         NaN   
53875128           72.0      NaN           NaN         NaN         2.7   
234941318           NaN      NaN           NaN         NaN         NaN   

          Team Fortress 2  ... Windforge Wooden Sen'SeY Ride! Carnival Tycoon  \
151603712             3.8  ..

In [52]:
utility.to_csv('./utility_matrix_simple.csv')